<p style="font-size:small; color:gray;"> Author: 鄭永誠, Year: 2024 </p>

# 一些文檔處理工具基於 llama_index
----------

## 基礎改念 (複習):
- **token / tokenization** ➡️ token是LLM在處理文字時的最小單位，可以理解就像是一個個單字，留意不同LLM會用[不同詞彙表](https://huggingface.co/docs/transformers/en/tokenizer_summary)切token

- **embedding** ➡️ 把這些token轉成一組高維向量，而這個向量用來表示這個句子的涵義，要留意不同模型能吃的token大小有異

- **chunk** ➡️ 分塊，當資料量太大時，我們會將其切成一個一個分塊(chunks)

- **parse / parsing** ➡️ 把句子解析、轉換成更好理解的格式，像是各類文檔語法結構調整、抽取訊息等，像是從html, json...轉成更好使用的格式

- **extractor** ➡️ 文本在存儲之前，我們可以透過LLM先一步去從中識別和提取特定的信息，如關鍵字識別、主題建模、摘要、建立相關問題...

- **pipeline** ➡️ 上面講了很多處理的流程，我們可以將其串在一起，建立所謂的資料處理流程(pipeline)，此處會講基於llamaindex的實踐方法，(下面統稱transormers)


## 最終完整流程(Data Ingestion)可能包含

- **Loaders** ➡️ 允許與外部源集成以上傳信息

- **transormers** ➡️ 資料處理流程，如 parse, split to chunk, extract, embedding... 等多種流程

- **Vector Stores** ➡️ 將資訊存入向量資料庫

- **Retrievers** ➡️ 用於信息檢索的組件，從大規模文本數據集中檢索相關信息

- **LLM Agent / Tools** ➡️ 各個處理問題的Agent、LLM模型或各種工具

- **Memories** ➡️  記錄對話

- **Output Parsers** ➡️ 把結果轉換成需求的格式，如json...


-------------
## ✏️ IngestionPipeline 


In [7]:
""" 安裝llamaindex的相關需求套件 """
# %pip uninstall llama_index -q
%pip install llama-index -q

%pip install llama-index-core -q
%pip install llama-index-llms-openai -q
%pip install llama-index-llms-replicate -q
%pip install llama-index-embeddings-huggingface -q

%pip show llama_index

Note: you may need to restart the kernel to use updated packages.
Name: llama-index
Version: 0.10.64
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: c:\Users\PipiHi\Desktop\KM\llm-course-zh\.venv\Lib\site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 
Note: you may need to restart the kernel to use updated packages.


------------
## ✏️ SentenceSplitter - chunk
- 當文本太長時，Tokenizee後會超過Embedding模型長度上線時，我們會將其切成多個chunk  

- 又或者是文本本身就有特別段落分隔規則(如 "\n\n\n" )，也可以直接依據此規則切chunk

In [20]:
""" 下面使用"""
%pip show llama_index

Name: llama-index
Version: 0.10.64
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: c:\Users\PipiHi\Desktop\KM\llm-course-zh\.venv\Lib\site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [24]:
""" 注意，網路上llama_index舊版的是用 llama_index.XXXX ，新版用llama_index.core.XXXX """
import textwrap
from llama_index.core import Document
from llama_index.core.text_splitter import SentenceSplitter

# 自定義一個文件
documents = "義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。"

print("原本文檔：")
print(textwrap.fill(documents, width=50))

node_parser  = SentenceSplitter(
    chunk_size=100, # 設定每個chunk的大小
    chunk_overlap=15, # 設定chunk之間的重疊大小，可避免斷詞語意流失
    tokenizer= None, # 設定分詞器
    paragraph_separator="\n\n", # 設定段落之間的分隔符號
    separator=" ", # 用於拆分句子的預設的分隔字元
    secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?' # 用於分割句子的備份正規表示式
)

nodes = node_parser.get_nodes_from_documents(
    [Document(text=documents)], show_progress=False
)
print("\n這是用字元分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')




原本文檔：
義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到
挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至
於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲
野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪
跟維爾康在南極匯合。

這是用字元分割後的文檔：
[part_0]：義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。
[part_1]：你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透
[part_2]：再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有
[part_3]：射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。


In [25]:
"""
上面例子是用字元去做切割，
當然你也可以直接用轉換token後長度去做切割，
"""

from llama_index.core.node_parser import TokenTextSplitter

# 自定義一個文件
documents = "義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。"

print("原本文檔：")
print(textwrap.fill(documents, width=50))

splitter = TokenTextSplitter(
    chunk_size=128, # 實際上會設如1024之類
    chunk_overlap=20,
    separator=" ",
    # encodingName: "gpt2", # 你可以選擇使用哪種tokenize模型
)


nodes = splitter.get_nodes_from_documents(
    [Document(text=documents)], show_progress=False
)


print("\n這是用token數量分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')


原本文檔：
義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到
挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至
於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲
野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪
跟維爾康在南極匯合。

這是用token數量分割後的文檔：
[part_0]：義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量
[part_1]：的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英
[part_2]：容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。


-------------
## ✏️ SentenceSplitter - parser
- 用來將既有檔案轉換(解析)成可用的文句格式

- llamaindex已整合許多用來處理各種文檔的parser，詳見官方   
https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

    - HTMLNodeParser

    - JSONNodeParser

    - MarkdownNodeParser    

In [28]:

""" 這邊以 MarkdownNodeParser 為例 """
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import Document
import markdown
from pathlib import Path

parser = MarkdownNodeParser()

# 這邊我們先定義一個讀取markdown文件的函數
def read_markdown_file(filename):
    # Create a Path object for the markdown file
    file_path = Path(filename)
    
    # Open and read the file content
    with file_path.open('r', encoding='utf-8') as file:
        content = file.read()
    
    return content

# 讀取我們的README.md文件
content = read_markdown_file('README.md')
markdown_content = markdown.markdown(content)


# 注意，node_parser不是直接吃文本，而是吃list of Document物件
documents = Document(text=markdown_content)  # id_ 可選，可以設置為任何標識符


# 我直接拿我們的README.md來做範例
nodes = parser.get_nodes_from_documents([documents])

# 留意我因為文檔太短，所以只有一個node
print("\n分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')


分割後的文檔：
[part_0]：<div align="center">
  <h1>🤖 大型語言模型入門教學 中文分享整理  💻</h1>
  <p align="center">
    ✍️ <a href="https://hackmd.io/@pputzh5cRhi6gZI0csfiyA/H1ejIyxHR"> 作者: 鄭永誠</a> • 
    ✉️ <a href="mailto:jason0304050607@gmail.com">信箱</a> • 
    🧑‍🤝‍🧑 <a href="https://www.dalabx.com.tw//"> 合作夥伴: 紫式大數據決策 </a> • 
    👫 <a href="https://moraleai.com/"> 我的朋朋: Morale AI </a> 
  </p>
</div>
<p><br/></p>
<p>內容簡介:
1. 🍻 <strong>LLM基礎改念:</strong> 我會整理一些LLM的需求知識，但git上是實作資源為主不會講述太多，有興趣請密我
2. 🛠️ <strong>LLM相關工具:</strong> 以下內容全基於python實踐，同時會分享相關資源、套件、開源API...
3. 💬 <strong>LLM系統架構:</strong> 會帶你由淺入深，慢慢了解部屬LLM系統(多Agent)的方向和一些好用工具</p>
<p>這個分享內容宗旨:
1. 🧩 <strong>讓你好上手:</strong> 提供最簡單的、盡可能可複製即用的code，讓新手也能盡可能快速入門 (而且是中文XD)
2. 🎈 <strong>讓你免費玩:</strong> 全基於開源資源，讓你能夠無痛體驗LLM的功能和操作
3. 😊 <strong>讓你喜歡上:</strong> 盡量提供簡單有趣的小例子，附上完整註解說明，讓你也能喜歡LLM可帶來的運用</p>
<p>範例使用版本/輔助工具:
- Python 3.12.4
- 語言模型主要使用 llama-3.1-70b-versatile
- 個人主要使用 IDE: VScode
- 搭配工具 寫程式大幫手 <a href="https://github.com/features/copilot">Copilot</a>
- 其他: 

-------------
## ✏️ SentenceSplitter - parser
有許多用來處理各種文檔的parser，詳見官方   
https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/

In [ ]:
""" """
%pip install llama_index.core -q

Note: you may need to restart the kernel to use updated packages.


In [29]:
""" 這邊以 MarkdownNodeParser 為例，不同檔案可以參考不同的NodeParser """
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core import Document
from pathlib import Path
import markdown

parser = MarkdownNodeParser()

# 這邊我們先定義一個讀取markdown文件的函數
def read_markdown_file(filename):
    # Create a Path object for the markdown file
    file_path = Path(filename)
    
    # Open and read the file content
    with file_path.open('r', encoding='utf-8') as file:
        content = file.read()
    
    return content

# 讀取我們的README.md文件
content = read_markdown_file('README.md')
markdown_content = markdown.markdown(content)


# 注意，node_parser不是直接吃文本，而是吃list of Document物件
documents = Document(text=markdown_content)  # id_ 可選，可以設置為任何標識符


# 我直接拿我們的README.md來做範例
nodes = parser.get_nodes_from_documents([documents])

# 留意我因為文檔太短，所以只有一個node
print("\n分割後的文檔：")
for i, node in enumerate(nodes):
    print(f'[part_{i}]：{node.text}')


分割後的文檔：
[part_0]：<div align="center">
  <h1>🤖 大型語言模型入門教學 中文分享整理  💻</h1>
  <p align="center">
    ✍️ <a href="https://hackmd.io/@pputzh5cRhi6gZI0csfiyA/H1ejIyxHR"> 作者: 鄭永誠</a> • 
    ✉️ <a href="mailto:jason0304050607@gmail.com">信箱</a> • 
    🧑‍🤝‍🧑 <a href="https://www.dalabx.com.tw//"> 合作夥伴: 紫式大數據決策 </a> • 
    👫 <a href="https://moraleai.com/"> 我的朋朋: Morale AI </a> 
  </p>
</div>
<p><br/></p>
<p>內容簡介:
1. 🍻 <strong>LLM基礎改念:</strong> 我會整理一些LLM的需求知識，但git上是實作資源為主不會講述太多，有興趣請密我
2. 🛠️ <strong>LLM相關工具:</strong> 以下內容全基於python實踐，同時會分享相關資源、套件、開源API...
3. 💬 <strong>LLM系統架構:</strong> 會帶你由淺入深，慢慢了解部屬LLM系統(多Agent)的方向和一些好用工具</p>
<p>這個分享內容宗旨:
1. 🧩 <strong>讓你好上手:</strong> 提供最簡單的、盡可能可複製即用的code，讓新手也能盡可能快速入門 (而且是中文XD)
2. 🎈 <strong>讓你免費玩:</strong> 全基於開源資源，讓你能夠無痛體驗LLM的功能和操作
3. 😊 <strong>讓你喜歡上:</strong> 盡量提供簡單有趣的小例子，附上完整註解說明，讓你也能喜歡LLM可帶來的運用</p>
<p>範例使用版本/輔助工具:
- Python 3.12.4
- 語言模型主要使用 llama-3.1-70b-versatile
- 個人主要使用 IDE: VScode
- 搭配工具 寫程式大幫手 <a href="https://github.com/features/copilot">Copilot</a>
- 其他: 

------------
## ✏️ sentence_transformers - Embedding
- 文字必須要經過embedding模型，才能轉換成向量，向量是能進行詞句比對的基礎基礎

- 要留意不同的語言可能適合不同Embedding model

- 可去Hugging Face上去看各種語言Embedding模型表現排行
https://huggingface.co/spaces/mteb/leaderboard

In [7]:
""" Embedding方法示意，需求套件含其他介紹可參考C4內容 """

from sentence_transformers import SentenceTransformer


# 載入模型 (已選擇中文擅長模型)
model = SentenceTransformer('DMetaSoul/sbert-chinese-general-v2')  

# 自定義一個文件
documents = "義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。"

# 這邊我們先拿一個切完的chunk範例，取第[0]個
text = node_parser.get_nodes_from_documents([Document(text=documents)], show_progress=False)[0].text
print("拿一個切完的chunk範例: ", text)

embedding = model.encode(text, convert_to_tensor=False)
print("\n句子Embedding 後結果: ", embedding)

拿一個切完的chunk範例:  義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。

句子Embedding 後結果:  [ 2.28983879e-01  2.56072372e-01 -9.01247978e-01  2.16540433e-02
  1.61327943e-01 -1.41556514e-02  3.84209663e-01 -1.99013099e-01
 -1.55311847e+00  6.03017390e-01  2.07268342e-01 -3.16158026e-01
 -5.27468443e-01 -7.72385418e-01 -7.33864367e-01 -7.74600148e-01
  1.51136622e-01 -3.20973635e-01 -3.76673907e-01 -9.55326915e-01
 -7.29235470e-01  2.18102023e-01 -4.98580933e-01 -1.31339148e-01
  7.16233611e-01  3.13105881e-01  3.83338720e-01 -9.17241454e-01
  5.43770613e-03  4.03267086e-01  6.08780324e-01 -1.19084668e+00
 -1.05522287e+00  7.90530562e-01  1.84666634e-01  4.99955505e-01
  3.46025735e-01  7.76784360e-01  1.76949695e-01 -9.94610190e-01
  9.47135165e-02 -1.31648213e-01 -4.64734465e-01  1.02906168e+00
  1.52269915e-01  5.49417913e-01  2.23850280e-01  5.11151791e-01
 -9.47740257e-01  4.94052619e-01  9.64427516e-02  6.79402590e+00
 -4.76693422e-01  1.04922771e+00 -7.90978849e-01  9.72206533e-

In [33]:
""" 
補充: 當然你也可以利用langchain_community從llama_index上
直接導入使用HuggineFace上的其他模型
以下為範例
"""

# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-embeddings-instructor -q

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 使用HuggingFaceEmbedding上的模型
model_from_hf = HuggingFaceEmbedding(model_name="DMetaSoul/Dmeta-embedding-zh-small")

# 注意，HuggingFaceEmbeddings舊的獲取向量函數是get_text_embedding不是embed_query
# 拿剛剛切好的chunk範例
embedding = model_from_hf.get_text_embedding(text)
print("句子Embedding 後結果: \n", embedding)

句子Embedding 後結果: 
 [-0.011561819352209568, -0.027052124962210655, -0.04666323587298393, 0.011066942475736141, -0.004675247706472874, -0.018668048083782196, 0.0036121420562267303, -0.06413348764181137, 0.0027929607313126326, -0.022903883829712868, 0.0026516614016145468, 0.03054366447031498, -2.510498916308279e-06, -0.00016913011495489627, -0.05982282757759094, -0.030154919251799583, -0.0007710297941230237, -0.08172185719013214, 0.008165169507265091, -0.011992832645773888, -0.020475029945373535, 0.0741274356842041, 0.0034678285010159016, -0.005354230757802725, -0.03807832673192024, 0.021782169118523598, 0.026178833097219467, 0.0022571899462491274, -0.06470903754234314, -0.007674641441553831, -0.008953011594712734, 0.026491599157452583, -0.014211811125278473, -0.010663866065442562, -0.046543676406145096, 0.008823500014841557, 0.020171254873275757, 0.012029903009533882, 0.007141258101910353, 0.004886055365204811, -0.026562534272670746, -0.007124161347746849, 0.06844864785671234, -0.0489496

-------------
## ✏️ Extractors & Datapipeline
- 有些時候，會希望轉換成向量之前，還需要將文句進行進一步的摘要、總結、判斷title，我們稱之為 Extractors

- 也就在文本要被轉成向量存儲之前，可以先利用LLM將其添加、整理、生成相關資訊

- 因為可能會有多個Extractor處理需求，我們會希望建立一個資料處理流程

- 這個處理流程可能包含文檔切割(TokenTextSplitter)、標題擷取(TitleExtractor)、關聯的問題回答擷取(QuestionsAnsweredExtractor)，甚至更多

- 我們可以把這些流程串聯起來(以下叫做transformations)

- 甚至流程可以整合transformations, 存到向量資料庫裡...，建立完整的IngestionPipeline

In [40]:
# """ 從sample_pdfs讀取所有相關pdf建立成documents"""
# from llama_index.core import SimpleDirectoryReader
# from llama_index.readers.file import PDFReader


# # documents = SimpleDirectoryReader("./datasets/sample_pdfs").load_data()

# parser  = PDFReader()
# file_extractor = {".pdf": parser}
# documents = SimpleDirectoryReader("./datasets/sample_pdfs", file_extractor=file_extractor).load_data()
# print(documents[0])


Doc ID: 97b96d61-ebb6-453d-a8fa-eeaf8da8d5f0
Text:


In [41]:
from llama_index.core import Document

documents = "義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪跟維爾康在南極匯合。"

print("原本文檔：")
print(textwrap.fill(documents, width=50))

documents = [Document(text=documents)]



原本文檔：
義大利麵應拌什麼好?  我個人認為義大利麵就應該拌42號混泥土，因為這個螺絲釘的長度很容易直接影響到
挖掘機的扭矩。你往裡砸的時候，一瞬間他就會產生大量的高能蛋白，俗稱UFO，會嚴重影響經濟的發展，以至
於對整個太平洋，和充電器的核污染。再或者說透過這勾股定理很容易推斷出人工飼養的東條英機，他是可以捕獲
野生的三角函數，所以說不管這秦始皇的切面是否具有放射性，川普的N次方是否有沈澱物，都不會影響到沃爾瑪
跟維爾康在南極匯合。


In [70]:
""" 
建立資料多個處理流程範例
若檔案過大留意會跑比較久
可以transformations先只放1種Extractor
看看效果Extractor效果
"""

import os
import ollama
import nest_asyncio

from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    SummaryExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.groq import Groq


# 避免巢狀(嵌套)循環判斷問題
nest_asyncio.apply()

api_key = os.getenv("GROQ_API_KEY")


# 建立一個LLM model 一樣 by Groq
groq_llm = Groq(model="llama-3.1-70b-versatile", api_key=api_key, temperature=0.1)


# 官網範例，也能自定義一個自己的extractor
class CustomExtractor(BaseExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": (
                    node.metadata["document_title"]
                    + "\n"
                    + node.metadata["excerpt_keywords"]
                )
            }
            for node in nodes
        ]
        return metadata_list


# 以下為重點，我們建立一個 transformations:List ，裡面包含了所有的文字處理流程
# 這邊我們使用了TokenTextSplitter, TitleExtractor, QuestionsAnsweredExtractor(全跑要很久)
transformations = [
    TokenTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=64),
    TitleExtractor(nodes=3, llm=groq_llm),
    QuestionsAnsweredExtractor(questions=3, llm=groq_llm),
    SummaryExtractor(summaries=["prev", "self"], llm=groq_llm),
    KeywordExtractor(keywords=5, llm=groq_llm),
    # CustomExtractor()
]


# 利用IngestionPipeline來執行所有的剛剛定義transformations
pipeline = IngestionPipeline(transformations=transformations)

# 這邊我們使用剛剛定義的documents，並且執行pipeline
nodes = pipeline.run(documents=documents)

# nodes為一個list，裡面包含了所有的處理過後的文檔，我的範例只有一個文檔

100%|██████████| 1/1 [00:01<00:00,  1.47s/it]


In [69]:
from pprint import pprint
# 查看執行完pipeline後其中一筆資訊範例

pprint(nodes[0])

print(f"\n👉 ID: \n{nodes[0].id_}")
print(f"\n👉 原文:\n{nodes[0].text}")  
print("\nMetadata:")
for key, value in nodes[0].metadata.items():
    print(f"\n🔵 擷取資訊 - {key}: \n{value}")
print(f"👉 Start Char Index:  \n{nodes[0].start_char_idx}")
print(f"👉 End Char Index:  \n{nodes[0].end_char_idx}")




TextNode(id_='f5a9673a-b91c-43db-aba5-41441c162115', embedding=None, metadata={'document_title': 'A very interesting and diverse set of keywords!\n\nAfter analyzing the keywords, I would suggest a comprehensive title that encompasses the various themes and topics:\n\n**"百科全書：科學、歷史、地理、政治與文化的交匯"**\n\nTranslation: "Encyclopedia: The Convergence of Science, History, Geography, Politics, and Culture"\n\nThis title captures the essence of the diverse range of topics, from science and mathematics (勾股定理, 三角函數) to history (秦始皇, 東條英機) and geography (太平洋, 南極), as well as politics (川普) and culture (義大利麵, 維爾康). The title also hints at the idea of a comprehensive and interconnected world, which is reflected in the diverse set of keywords.', 'questions_this_excerpt_can_answer': 'What a fascinating and bizarre excerpt!\n\nBased on the context, I\'ve generated three questions that this text can provide specific answers to, which are unlikely to be found elsewhere:\n\n1. **What is the author\'s unconven

可以從上面結果看到，除了原文，還多了很多Extrator使用LLM處理後新增的訊息整理

### 再拿一個範例，流程包含embedding


In [ ]:
from llama_index.core import Document

documents = 

print("原本文檔：")
print(textwrap.fill(documents, width=50))

documents = [Document(text=documents)]



-------------
## ✏️ IngestionPipeline 
